In [21]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [22]:
class MyDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)
        self.x = np.array(self.data.iloc[:, :-1])
        self.y = np.array(self.data.iloc[:, -1])
        
    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.float32), torch.tensor(self.y[index], dtype=torch.long)
        
    def __len__(self):
        return len(self.data)


In [23]:
model = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# L2 Regülerizasyonu (Weight Decay):


In [24]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x
    
# L2 Regülerizasyonu
l2_reg = 0.01
for param in model.parameters():
    param.data -= l2_reg * param.data.norm(2)

In [25]:
train_dataset = MyDataset('cure_the_princess_train.csv')
val_dataset = MyDataset('cure_the_princess_validation.csv')
test_dataset = MyDataset('cure_the_princess_test.csv')


In [26]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [27]:
def train(model, train_dataloader, optimizer, loss_fn):
    model.train()
    for batch, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()


In [28]:
def evaluate(model, val_dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_dataloader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total


In [29]:
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    acc = evaluate(model, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', acc)

Epoch: 1 | Validation Accuracy: 49.681528662420384
Epoch: 2 | Validation Accuracy: 49.681528662420384
Epoch: 3 | Validation Accuracy: 49.681528662420384
Epoch: 4 | Validation Accuracy: 49.681528662420384
Epoch: 5 | Validation Accuracy: 49.681528662420384
Epoch: 6 | Validation Accuracy: 49.681528662420384
Epoch: 7 | Validation Accuracy: 49.681528662420384
Epoch: 8 | Validation Accuracy: 51.910828025477706
Epoch: 9 | Validation Accuracy: 64.64968152866243
Epoch: 10 | Validation Accuracy: 72.61146496815287


In [20]:
test_acc = evaluate(model, test_dataloader)
print('Test Accuracy:', test_acc)

Test Accuracy: 50.259067357512954


In [ ]:
# Dropout Regülerizasyonu

In [30]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(100, 50)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)
        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [31]:
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    acc = evaluate(model, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', acc)

Epoch: 1 | Validation Accuracy: 77.38853503184713
Epoch: 2 | Validation Accuracy: 80.89171974522293
Epoch: 3 | Validation Accuracy: 85.03184713375796
Epoch: 4 | Validation Accuracy: 87.57961783439491
Epoch: 5 | Validation Accuracy: 89.80891719745223
Epoch: 6 | Validation Accuracy: 91.71974522292993
Epoch: 7 | Validation Accuracy: 92.03821656050955
Epoch: 8 | Validation Accuracy: 89.171974522293
Epoch: 9 | Validation Accuracy: 91.0828025477707
Epoch: 10 | Validation Accuracy: 91.0828025477707


In [32]:
test_acc = evaluate(model, test_dataloader)
print('Test Accuracy:', test_acc)

Test Accuracy: 88.08290155440415
